In [2]:
import pandas as pd

df = pd.read_csv('../Data Collection/jobs_data.csv')
df.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,Job Description,Work from home,Salary,Schedule type,Qualifications,Job ID,Search Date,Search Query
0,Austria,Europe,True,True,default,google.com,Android Developer,Agentur LOOP New Media GmbH,Anywhere,"Indeed, Jooble, Jobs By Workable, Trabajo.org ...","LOOP is a digital-first lead agency, exploring...",True,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIERldmVsb3BlciIsIm...,2025-01-13 12:08:48 UTC,Android developer
1,Austria,Europe,True,True,default,google.com,Android Kotlin Developer*,BAWAG Group,"Vienna, Austria","Karriere.at, XING, Indeed, LinkedIn, Jooble, J...",Android Kotlin Developer*\n\nBAWAG Group is th...,NaN,"€44,476.88 a year",Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIEtvdGxpbiBEZXZlbG...,2025-01-13 12:08:48 UTC,Android developer
2,Austria,Europe,True,True,default,google.com,"Software Engineer, Android",Bitpanda,"Vienna, Austria",Startup Jobs,Who we are\n\nWe simplify wealth creation. Fou...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJTb2Z0d2FyZSBFbmdpbmVlciwgQW...,2025-01-13 12:08:48 UTC,Android developer
3,Austria,Europe,True,True,default,google.com,Android Application Development,Eastern Techno Solutions Pty,Austria,"Trabajo.org - Stellenangebote, Arbeit",Android Developer Job Description:We are looki...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIEFwcGxpY2F0aW9uIE...,2025-01-13 12:08:48 UTC,Android developer
4,Austria,Europe,True,True,default,google.com,Android & iOS Developer,ventopay,"Hagenberg, Austria","Ventopay, Informatikjobs.at, XING, Expertini",Was sind deine Aufgaben?\n• Du gestaltest attr...,NaN,NaN,Part-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIFx1MDAyNiBpT1MgRG...,2025-01-13 12:08:48 UTC,Android developer
